# Test de YOLO sur raspberry pi


## <span style="color:lightblue">Installation des dépendances</span>


In [ ]:
%pip install -q -r requirements.txt

## <span style="color:lightblue">Importation des dépendances</span>


In [2]:
from ultralytics import YOLO

## <span style="color:lightblue">Code principal</span>


### <span style="color:lightgreen">Environnement</span>


In [3]:
# formats = ["onnx", "openvino", "edgetpu", "ncnn", "engine"]
# models = ["yolov8n_trained", "yolo11n_trained"]

IMAGE_SIZE = 640

### <span style="color:lightgreen">Chargement des modèles</span>


In [ ]:
modelv8n = YOLO(f"./saved/yolov8n_trained.pt")
model = YOLO(f"./saved/yolo11n_trained.pt")

### <span style="color:lightgreen">Export des modèles sous onnx</span>


Le type de device pour hailo est inconnu.


In [ ]:
modelv8n.export(format="onnx", imgsz=IMAGE_SIZE, half=True, device="cpu")

In [ ]:
model.export(format="onnx", imgsz=IMAGE_SIZE, half=True, device="cpu")

Techniquement on pourrait mettre le batch pour faire du simultané mais appart pour hailo on va partir du principe que l'on va juste envoyer image par image (à faire sous linux).


In [ ]:
model.export(format="onnx", imgsz=IMAGE_SIZE, half=True, device=0)

### <span style="color:lightgreen">Export de yolo11n sous openvino</span>


Sans rien c'est mieux que int8, mais half fait gagner 1ms d'inférence.


In [ ]:
model.export(format="openvino", imgsz=IMAGE_SIZE, half=True, device="cpu")

### <span style="color:lightgreen">Export de yolo11n sous ncnn</span>


In [ ]:
model.export(format="ncnn", imgsz=IMAGE_SIZE, half=True, device="cpu")

### <span style="color:lightgreen">Export de yolo11n sous edgetpu (à faire sous linux)</span>


Pas d'autres arguments possibles.


In [ ]:
model.export(format="edgetpu", imgsz=IMAGE_SIZE)

### <span style="color:lightgreen">Export de yolo11n sous engine (à faire sous jetson)</span>


Pas encore testé.


In [ ]:
model.export(format="engine", imgsz=IMAGE_SIZE, int8=True, device="dla:0", data="../data/yolodataset4/data.yaml")

## <span style="color:lightblue">Sources</span>


- https://docs.ultralytics.com/fr/modes/export/
